# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df["Max Temp"]= city_data_df["Max Temp"]-273.15
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,xiaoweizhai,26.1903,107.5125,5.17,88,100,2.56,CN,1699848179
1,1,udachny,66.4167,112.4000,-11.97,84,100,3.64,RU,1699848104
2,2,chifeng,42.2683,118.9636,0.69,47,0,3.56,CN,1699848179
3,3,sardarshahr,28.4333,74.4833,22.09,39,2,2.26,IN,1699848180
4,4,ushuaia,-54.8000,-68.3000,9.81,53,0,4.12,AR,1699848084


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    color="City",
    alpha= 0.5,
    tiles= "ESRI",
    scale= 1
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]


# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,port mathurin,-19.6833,63.4167,25.17,78,0,2.01,MU,1699848067
79,79,tura,25.5198,90.2201,25.91,56,0,0.60,IN,1699848104
106,106,besisahar,28.2313,84.3761,22.33,51,0,2.12,NP,1699848202
140,140,maceio,-9.6658,-35.7353,21.69,100,0,1.54,BR,1699848015
154,154,vila velha,-20.3297,-40.2925,25.97,89,0,3.60,BR,1699848212
239,239,palmas,-10.2128,-48.3603,26.93,74,0,1.54,BR,1699848232
266,266,aswan,24.0934,32.9070,24.61,60,0,2.57,EG,1699848236
289,289,salalah,17.0151,54.0924,26.77,52,0,2.05,OM,1699848241
391,391,coremas,-7.0144,-37.9458,26.26,52,0,3.30,BR,1699848268
510,510,tsiombe,-25.3000,45.4833,25.15,59,0,3.96,MG,1699848294


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
24,port mathurin,MU,-19.6833,63.4167,78,
79,tura,IN,25.5198,90.2201,56,
106,besisahar,NP,28.2313,84.3761,51,
140,maceio,BR,-9.6658,-35.7353,100,
154,vila velha,BR,-20.3297,-40.2925,89,
239,palmas,BR,-10.2128,-48.3603,74,
266,aswan,EG,24.0934,32.9070,60,
289,salalah,OM,17.0151,54.0924,52,
391,coremas,BR,-7.0144,-37.9458,52,
510,tsiombe,MG,-25.3000,45.4833,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port mathurin - nearest hotel: Escale Vacances
tura - nearest hotel: No hotel found
besisahar - nearest hotel: Hotel Tulsi
maceio - nearest hotel: Maceió Hostel e Pousada
vila velha - nearest hotel: Hotel Prainha
palmas - nearest hotel: Hotel Triângulo Mineiro
aswan - nearest hotel: Yaseen hotel
salalah - nearest hotel: Muscat International Hotel
coremas - nearest hotel: Pousada Vitória
tsiombe - nearest hotel: No hotel found
topolobampo - nearest hotel: Hotel Marina Topolobampo


,City,Country,Lat,Lng,Humidity,Hotel Name
24,port mathurin,MU,-19.6833,63.4167,78,Escale Vacances
79,tura,IN,25.5198,90.2201,56,No hotel found
106,besisahar,NP,28.2313,84.3761,51,Hotel Tulsi
140,maceio,BR,-9.6658,-35.7353,100,Maceió Hostel e Pousada
154,vila velha,BR,-20.3297,-40.2925,89,Hotel Prainha
239,palmas,BR,-10.2128,-48.3603,74,Hotel Triângulo Mineiro
266,aswan,EG,24.0934,32.9070,60,Yaseen hotel
289,salalah,OM,17.0151,54.0924,52,Muscat International Hotel
391,coremas,BR,-7.0144,-37.9458,52,Pousada Vitória
510,tsiombe,MG,-25.3000,45.4833,59,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot

map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    scale=1,
    color="City",
    alpha= 0.5,
    tiles="ESRI",
    hover_cols=["Hotel Name", "Country"]
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)